In [1]:
from gurobipy import *
import gurobipy as gp

# type declaration
from typing import Dict, List


import pandas as pd

import numpy as np

# %load_ext nb_black
# %load_ext lab_black


import plotly.express as px

import plotly.graph_objects as go
%matplotlib inline

# Q1

In [2]:
dict_swimmers = {("N", "BA") : 40.1, ("N", "BR") : 41.7, ("N", "BU") : 41.1, ("N", "FR") : 33.3,
("M", "BA") : 39.6, ("M", "BR") : 37.6, ("M", "BU") : 31.9, ("M", "FR") : 31.2,
("L", "BA") : 35.3, ("L", "BR") : 37.9, ("L", "BU") : 36.2, ("L", "FR") : 33.1, 
("A", "BA") : 30.6, ("A", "BR") :  34.1, ("A", "BU") : 33.3, ("A", "FR") :  29.1,
("C", "BA") : 36.9, ("C", "BR") :  37.2, ("C", "BU") : 29.0, ("C", "FR") :  31.6}

dict_swimmers_obj = [("N", "BA") , ("N", "BR") , ("N", "BU") , ("N", "FR") ,
("M", "BA") , ("M", "BR") , ("M", "BU") , ("M", "FR"),
("L", "BA"), ("L", "BR"), ("L", "BU"), ("L", "FR"), 
("A", "BA"), ("A", "BR"), ("A", "BU"), ("A", "FR"),
("C", "BA"), ("C", "BR"), ("C", "BU"), ("C", "FR")]
names_list = ["N", "M", "L", "A", "C"]
types_lanes = ["BA", "BR", "BU", "FR"]

In [3]:
model = Model()

x = model.addVars(dict_swimmers, obj = dict_swimmers, vtype = GRB.BINARY, name = "swimmers")

model.ModelSense=gp.GRB.MINIMIZE


model.addConstrs(gp.quicksum((x[(i,j)] for j in types_lanes)) <= 1 for i in names_list)

for j in types_lanes:
    model.addConstr(gp.quicksum((x[(i,j)] for i in names_list)) == 1)   

model.optimize()
print("optimal value",model.objval)
model.printAttr("X")


Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-10
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9 rows, 20 columns and 40 nonzeros
Model fingerprint: 0x69b34e51
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 136.3000000
Presolve time: 0.00s
Presolved: 9 rows, 20 columns, 40 nonzeros
Variable types: 0 continuous, 20 integer (20 binary)
Found heuristic solution: objective 135.8000000

Root relaxation: objective 1.287000e+02, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work


# Q2

Specifically, for a flight one month from today, there are 100 seats available.

Customers that attempt to book a ticket in the last 10 days are in general willing to pay a higher price.

Thus, to maximize its revenues, the company charges a price of $150 during period 1 (first 20 days until departure) and a price of $500 during period 2.

The company must set a booking limit α, on the maximum number of tickets it will accept to sell during the first period.

If it knew that exactly x customers will arrive during period 2, of course it would set the booking limit to 100 −x.

The issue is that the number of customers cannot be known a priori.

The company assumes that the number of lower paying customers that arrive in period 1 and the number of high paying customers that arrive in period 2 follow Poison distributions with λ1 = 75 and λ2 = 50 respectively


# Q2a Standard Capacity allocation problem

Use simulation to determine the optimal booking policy to maximize expected revenue: for every booking limit between 20 and 80, simulate 10000 paths and compute the expected revenue. Plot the expected revenue as a function of the booking limit. (30 points)





In [4]:
n = 10000
max = 100
booking_limit = list(range(20,81))
customerP1 = np.random.poisson(lam=75,size=10000)
customerP2 = np.random.poisson(lam=50,size=10000)
profitAlpha = []

for i in booking_limit:
    salesP1 = np.where(customerP1 <= i, customerP1, i)

    ticketsLeft = max - salesP1

    salesP2 = np.where(customerP2 > ticketsLeft, ticketsLeft, customerP2)
    
    revenue = (150 * salesP1 + 500 * salesP2)
    profitAlpha.append([i, np.mean(revenue)])



In [5]:
profitAlpha = pd.DataFrame(profitAlpha, columns = ["Alpha", "ER"])
profitAlpha.sort_values("ER")

,Alpha,ER
60,80,24231.200
59,79,24335.325
58,78,24454.465
57,77,24588.410
56,76,24739.785
...,...,...
24,44,31244.950
28,48,31265.320
25,45,31284.450
27,47,31296.800


In [6]:
fig = go.Figure()
fig.add_trace(
    go.Scatter( x = profitAlpha['Alpha'], y = profitAlpha['ER'],
        # name="Revenue"
    ))
fig.add_trace(
    go.Bar(
        x=profitAlpha['Alpha'],
        y=profitAlpha['ER'],
        # name="Profit"
    ))
fig.show()

In [7]:
px.line(profitAlpha, x = 'Alpha', y = 'ER')

In [8]:
px.bar(profitAlpha, x = 'Alpha', y = 'ER')

# Q2b

Specifically, for a flight one month from today, there are 100 seats available.

Customers that attempt to book a ticket in the last 10 days are in general willing to pay a higher price.

Thus, to maximize its revenues, the company charges a price of $150 during period 1 (first 20 days until departure) and a price of $500 during period 2.

The company must set a booking limit α, on the maximum number of tickets it will accept to sell during the first period.

If it knew that exactly x customers will arrive during period 2, of course it would set the booking limit to 100 −x.

The issue is that the number of customers cannot be known a priori.

The company assumes that the number of lower paying customers that arrive in period 1 and the number of high paying customers that arrive in period 2 follow Poison distributions with λ1 = 75 and λ2 = 50 respectively


In [9]:
import numpy as np

In [10]:
n = 10000
max = 100
booking_limit_alpha = list(range(20,81))

compensation = list(range(100,301))

customerP1 = np.random.poisson(lam=75,size=10000)
customerP2 = np.random.poisson(lam=50,size=10000)
profitAlpha2 = []

for i in booking_limit_alpha:
    salesP1 = np.where(customerP1 <= i, customerP1, i)
    
    for p in compensation: 
        
        prob = np.random.binomial(salesP1, ((p-100)/200), 10000)
        ticketsLeft = max - salesP1 + prob
        salesP1 -= prob

        salesP2 = np.where(customerP2 > ticketsLeft, ticketsLeft, customerP2)
        revenue = (150 * salesP1 + 500 * salesP2 - prob * p)
        
        profitAlpha2.append([i, p, np.mean(revenue)])

    


In [11]:
results = []

demand_p1 = np.random.poisson(lam=75,size=10000)
demand_p2 = np.random.poisson(lam=50,size=10000)
parameter_p = list(range(100,301))
alpha = list(range(20,81))
n_simulations = 10000
seats_available = 100
price_p1 = 150
price_p2 = 500


for a in alpha:
    for p in parameter_p:
        paying_customers_p1 = np.where(demand_p1 <= 0, demand_p1, a)

        # compute paying customers in p1
        prefer_refund = np.random.binomial(paying_customers_p1, ((p-100)/200), n_simulations)

        # compute paying customers in p2
        remaining_seats = seats_available - paying_customers_p1

        p2_seat_shortage = np.where(remaining_seats < demand_p2, demand_p2 - remaining_seats, 0)
        p1_refunds = np.where(prefer_refund >= p2_seat_shortage, p2_seat_shortage, prefer_refund)

        cost_of_refunds = p1_refunds * p

        paying_customers_p1_updated = paying_customers_p1 - p1_refunds
        remaining_seats_updated = remaining_seats + p1_refunds

        paying_customers_p2 = np.where(demand_p2 > remaining_seats_updated, remaining_seats_updated, demand_p2)

        # compute expected revenue
        expected_revenue = ((paying_customers_p1 * price_p1) - (cost_of_refunds)) + (paying_customers_p2 * price_p2)

        # save resuitls to list
        results.append([a,p,np.mean(expected_revenue)])





In [12]:
profitAlpha2 = pd.DataFrame(profitAlpha2, columns = ["Alpha", "Refund Tickets", "ER"])
profitAlpha2.sort_values("ER")




,Alpha,Refund Tickets,ER
12060,80,100,24236.4350
12061,80,101,24325.6206
11859,79,100,24341.7150
11860,79,101,24433.2420
11658,78,100,24461.3800
...,...,...,...
5628,48,100,31222.7450
5428,47,101,31226.7262
5025,45,100,31231.1000
5427,47,100,31249.8550


In [13]:
profitAlpha2[profitAlpha2.Alpha == 47]

,Alpha,Refund Tickets,ER
5427,47,100,31249.8550
5428,47,101,31226.7262
5429,47,102,31195.5178
5430,47,103,31156.6936
5431,47,104,31103.3934
...,...,...,...
5623,47,296,24999.6000
5624,47,297,24999.6000
5625,47,298,24999.6000
5626,47,299,24999.6000


In [14]:
profitAlpha2 = profitAlpha2.groupby(by = "Alpha").agg({"ER": "mean"}).reset_index()


In [15]:
profitAlpha2[profitAlpha2.Alpha == 20]

,Alpha,ER
0,20,25247.135391


#### Plot the optimal revenew as a function of the optimal booking limit assuming that teh refund price is set optimally for the specific booking limit



In [16]:
fig = go.Figure()
fig.add_trace(
    go.Scatter( x = profitAlpha['Alpha'], y = profitAlpha['ER'],
        # name="Revenue"
    ))
fig.add_trace(
    go.Bar(
        x=profitAlpha2['Alpha'],
        y=profitAlpha2['ER'],
        # name="Profit"
    ))
fig.show()

## q3 A

In [49]:
Scenaria = ["A", "B", "C"]

probs = {"A": 0.3, "B": 0.2, "C": 0.5}

demands = {("A", "1") : 5, ("A", "2") : 3, ("A", "3") : 7.8, ("A", "4") : 4,
("B", "1") : 3, ("B", "2") : 1, ("B", "3") : 6, ("B", "4") : 3,
("C", "1") : 4.2, ("C", "2") : 4.1, ("C", "3") : 3, ("C", "4") : 5.4, }

distribution_centers = ("1", "2", "3", "4")




production_cost = 10

profit = 20

warehouses = ("WA", "WB")

transport_warehouse = {"WA": 1, "WB": 1.5}
transportation_costs = {("WA", "1") : 1.5, ("WA", "2") : 2, ("WA", "3") : 2, ("WA", "4"): 3,
("WB", "1") : 2, ("WB", "2") : 1.2, ("WB", "3") : 1.1, ("WB", "4"): 2.5}

model = gp.Model()

# first stage model
p = model.addVars(warehouses, vtype = gp.GRB.INTEGER, name = "prodcution")

# second stage model
d_wdc = model.addVars(warehouses, distribution_centers, Scenaria, vtype = gp.GRB.INTEGER, name = "transport")
inventory_d = model.addVars(distribution_centers, Scenaria, vtype = gp.GRB.INTEGER, name = "inventory")

sold_d = model.addVars(distribution_centers, Scenaria, vtype = gp.GRB.INTEGER, name = "sold")

model.setObjective(gp.quicksum(profit * probs[s] * sold_d[d,s] for s in Scenaria for d in distribution_centers) 
- gp.quicksum(d_wdc[w,d,s] * transportation_costs[w,d]* probs[s] for s in Scenaria for w in warehouses for d in distribution_centers) 
- gp.quicksum(p[w]*(production_cost + transport_warehouse[w]) for w in warehouses   ))


model.addConstrs((gp.quicksum(d_wdc[w,d,s]    for d in distribution_centers    )   <= p[w] for w in warehouses for s in Scenaria ), name = "receive_constraint")
model.addConstrs((inventory_d[d, s] == gp.quicksum(d_wdc[w,d,s] for w in warehouses) for d in distribution_centers for s in Scenaria), name = "distro_receive_sent_limit")


model.addConstrs((sold_d[d,s] <= inventory_d[d, s] for d in distribution_centers for s in Scenaria), name = "sold_constraint")


model.addConstrs((sold_d[d,s] <= demands[s, d] * 1000 for d in distribution_centers for s in Scenaria), name = "sold_constraint")

model.ModelSense = gp.GRB.MAXIMIZE
model.optimize()
print("Optimal Value:", model.Objval )

model.printAttr("X")


Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 42 rows, 50 columns and 102 nonzeros
Model fingerprint: 0xcb51c001
Variable types: 0 continuous, 50 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 8e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 24 rows and 12 columns
Presolve time: 0.00s
Presolved: 18 rows, 38 columns, 66 nonzeros
Variable types: 0 continuous, 38 integer (0 binary)

Root relaxation: objective 1.049990e+05, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0        

## q3 b

In [113]:

transport_costs = pd.read_csv("/home/angelo/Documents/Uni/Courses/Management Science/MM_Code_repo/data/TranportationCosts.csv")
demands = pd.read_csv("/home/angelo/Documents/Uni/Courses/Management Science/MM_Code_repo/data/demand.csv")

Scenaria = list(demands.columns)[1:]

probs={}
for i in range(0, len(Scenaria)):
    probs[Scenaria[i]] = 1/100


production_cost = 10
profit = 20
fixedcosts = 10**5

demand = {}
distribution_centers = list(demands.loc[:, "DistributionCenter"])

for d in range(0, len(demands)):
    for s in range(1, len(demands.columns)):
        key = (distribution_centers[d], Scenaria[s-1])
        value = demands.loc[d, Scenaria[s-1]]
        demand[key] = value 



warehouses = list (transport_costs.loc[:, "Warehouse"])
transport_warehouse={}
for w in range(0, len(transport_costs)):
    key = warehouses[w]
    value = transport_costs.loc[w, "PFacility"]
    transport_warehouse[key] = value

transport_costs = {}
for w in range(0, len(transport_costs)):
    for d in range(2, len(transport_costs.columns)): 
        key = (warehouses[w], distribution_centers[d- 2])
        value = transport_costs.loc[w, distribution_centers[d - 2]]
        transportation_costs[key] = value 


model = gp.Model()

model.ModelSense = gp.GRB.MAXIMIZE
# first stage 

p = model.addVars(warehouses, vtype = gp.GRB.INTEGER, name = "prodcution")
x = model.addVars(warehouses, vtype = gp.GRB.BINARY, name = "investment")

# second stage 

d_wdc = model.addVars(warehouses, distribution_centers, Scenaria, 
                        vtype = gp.GRB.INTEGER, name = "transport")
                        
inventory_d = model.addVars(distribution_centers, Scenaria,
                             vtype = gp.GRB.INTEGER, name = "inventory")

sold_d = model.addVars(distribution_centers, Scenaria, 
                        vtype = gp.GRB.INTEGER, name = "sold")


model.setObjective(gp.quicksum(profit * probs[s] * sold_d[d,s] for s in Scenaria for d in distribution_centers) 
                    - gp.quicksum(d_wdc[w,d,s] * transportation_costs[w,d]* probs[s] for s in Scenaria for w in warehouses for d in distribution_centers) 
                    - gp.quicksum(p[w]*(production_cost + transport_warehouse[w]) for w in warehouses   )
                    - gp.quicksum(x[w] * fixedcosts for w in warehouses)
                    )

model.addConstrs((gp.quicksum(d_wdc[w,d,s]    for d in distribution_centers    )   <= p[w]*x[w] for w in warehouses for s in Scenaria ), name = "sent_production_limit")

model.addConstrs((inventory_d[d,s] == gp.quicksum( d_wdc[w,d,s] for w in warehouses) for d in distribution_centers for s in Scenaria),
                    name="distro_receive_sent_limit")

model.addConstrs((sold_d[d, s] <= inventory_d[d, s] * 1000 for d in distribution_centers for s in Scenaria), name = "sent_production_limit"   )

model.addConstrs((sold_d[d, s] <= demands[d, s] * 1000 for d in distribution_centers for s in Scenaria), name = "sent_production_limit"   )


model.optimize()
print("Optimal Value:", model.Objval )

model.printAttr("X")








KeyError: ('WH-Seville', 'DC-Vienna')

In [114]:
Scenaria

['S1',
 'S2',
 'S3',
 'S4',
 'S5',
 'S6',
 'S7',
 'S8',
 'S9',
 'S10',
 'S11',
 'S12',
 'S13',
 'S14',
 'S15',
 'S16',
 'S17',
 'S18',
 'S19',
 'S20',
 'S21',
 'S22',
 'S23',
 'S24',
 'S25',
 'S26',
 'S27',
 'S28',
 'S29',
 'S30',
 'S31',
 'S32',
 'S33',
 'S34',
 'S35',
 'S36',
 'S37',
 'S38',
 'S39',
 'S40',
 'S41',
 'S42',
 'S43',
 'S44',
 'S45',
 'S46',
 'S47',
 'S48',
 'S49',
 'S50',
 'S51',
 'S52',
 'S53',
 'S54',
 'S55',
 'S56',
 'S57',
 'S58',
 'S59',
 'S60',
 'S61',
 'S62',
 'S63',
 'S64',
 'S65',
 'S66',
 'S67',
 'S68',
 'S69',
 'S70',
 'S71',
 'S72',
 'S73',
 'S74',
 'S75',
 'S76',
 'S77',
 'S78',
 'S79',
 'S80',
 'S81',
 'S82',
 'S83',
 'S84',
 'S85',
 'S86',
 'S87',
 'S88',
 'S89',
 'S90',
 'S91',
 'S92',
 'S93',
 'S94',
 'S95',
 'S96',
 'S97',
 'S98',
 'S99',
 'S100']

In [105]:
transportation_costs[w,d]

KeyError: (22, 40)

In [ ]:
(distribution_centers[d], Scenaria[s-1])